In [6]:
# ============================================
# ProgettoDL_FootballSegmentation — YOLOv8 su frame estratti
# Notebook: inferenza YOLOv8 su frame già presenti + ricomposizione video
# Pipeline: (1) Setup → (2) Percorsi → (3) Caricamento modello → (4) Selezione cartella frame
#           → (5) Inferenza → (6) Salvataggio frame + ricomposizione video
# ============================================


# ---------- (1) Setup e import librerie ----------
from pathlib import Path
import sys, re, time
import cv2
import torch
from ultralytics import YOLO
import tkinter as tk
from tkinter import filedialog
from IPython.display import display, Markdown

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x, **k: x  # fallback senza barra di progresso

# ---------- (2) Definizione dei percorsi ----------
# BASE_DIR: cartella del notebook (01_yolov8 / 1_yolov8)
BASE_DIR = Path.cwd()
# data_inputs nella root del progetto (fratello di 01_yolov8)
DATA_INPUTS_DIR = BASE_DIR.parent / "data_inputs"
# Cartelle accanto al notebook YOLOv8
MODELS_DIR = BASE_DIR / "models"
DATA_OUTPUTS_DIR = BASE_DIR / "data_outputs"
MODELS_DIR.mkdir(parents=True, exist_ok=True)
DATA_OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

# Import utilità di postprocessing (presenti nella root del progetto)
sys.path.append(str(BASE_DIR.parent))
from utils.preprocessing import natural_sort
from utils.postprocessing import save_image  # (usiamo cv2 per il video per mostrare la barra)


# ---------- (3) Caricamento modello (Versione 1: robusta) ----------------------------------------------------------------------
model_path = MODELS_DIR / "yolov8x-seg.pt"
if not model_path.exists():
    # Forza il download (Ultralytics salva tipicamente nella working dir o talvolta nella dir del notebook)
    _ = YOLO("yolov8x-seg.pt")
    # Cerca il file in più posizioni "probabili" e spostalo in models/
    for cand in [Path("yolov8x-seg.pt"), BASE_DIR / "yolov8x-seg.pt"]:
        if cand.exists():
            cand.replace(model_path)
            break

# Istanzia il modello dai pesi locali
model = YOLO(model_path)

# Se disponibile CUDA, usa GPU 0; altrimenti CPU
device = 0 if torch.cuda.is_available() else "cpu"
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available(), "| Device:", device)
if torch.cuda.is_available():
    print("GPU usata:", torch.cuda.get_device_name(0))

# Per il report: tipo architettura YOLO (derivato dal nome del peso)
yolo_type = model_path.stem  # es. "yolov8x-seg"

# ---------- (4) Selezione cartella frame da analizzare con finestra ---------------------------------------------------------------------
root = tk.Tk()
root.withdraw()  # nasconde la finestra principale
frames_dir = Path(filedialog.askdirectory(title="Seleziona la cartella dei frame estratti"))
if not frames_dir.exists() or not frames_dir.is_dir():
    raise FileNotFoundError("Cartella frame non trovata o non valida.")

# Prende tutti i frame PNG con prefisso 'frame' e li ordina in modo naturale
frame_paths = natural_sort(frames_dir.glob("frame*.png"))

if not frame_paths:
    raise RuntimeError(f"Nessun frame PNG trovato in {frames_dir}")

# Estrai automaticamente l'ID clip (xx) dal nome cartella (es. .../extracted_frames_clip01). Default '00'
m = re.search(r"clip(\d+)", frames_dir.name)
clip_id = m.group(1) if m else "00"


# ---------- (5) Inferenza YOLOv8 sui frame ----------
# FPS fisso per la ricostruzione del video
fps = 25.0

# Cartella per i frame segmentati in 01_yolov8/data_outputs:
# "segmented_frames_clipxx_yolov8"
seg_frames_dir = DATA_OUTPUTS_DIR / f"segmented_frames_clip{clip_id}_yolov8"
seg_frames_dir.mkdir(parents=True, exist_ok=True)

segmented_frames = []   # terrà gli array RGB annotati, nell'ordine corretto
processed_names = []    # per la lista dinamica mostrata sotto la barra

# Display dinamico della lista sotto la barra (mostriamo gli ultimi 50 file)
list_handle = display(Markdown("_Inizio inferenza…_"), display_id=True)

# (Aggiunta timing totale inferenza)
start_time = time.time()  # Per misurare la durata totale dell'inferenza

for i, frame_path in tqdm(list(enumerate(frame_paths, start=1)), desc="Completamento inferenza YOLOv8 sui frames", unit="frame"):
    # Carica frame e converte in RGB
    frame_bgr = cv2.imread(str(frame_path))
    if frame_bgr is None:
        continue
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # Predizione con YOLOv8 (GPU se disponibile)
    results = model.predict(frame_rgb, device=device, verbose=False)

    # Immagine annotata (maschere/box/label): array RGB
    annotated = results[0].plot()

    # Salvataggio frame segmentato
    seg_name = f"frame{i:02d}_clip{clip_id}_segmented_yolov8.png"
    seg_path = seg_frames_dir / seg_name
    save_image(annotated, str(seg_path))

    # Accumula in memoria per ricostruzione video
    segmented_frames.append(annotated)

    # Aggiorna lista dinamica (ultimi 10)
    processed_names.append(frame_path.name)
    last = processed_names[-10:]
    list_handle.update(Markdown(
        "**Frame analizzati (ultimi 10):**  \n" + "<br>".join(last) + f"<br><br>Totale: **{len(processed_names)}**"
    ))

if not segmented_frames:
    raise RuntimeError("Nessun frame segmentato generato.")

# Calcolo tempo totale inferenza (fine Sezione 5)
elapsed_time = time.time() - start_time


# ---------- (6) Ricomposizione video (barra + lista dinamica coerenti) ----------
# Salva il video in 01_yolov8/data_outputs come "segmented_clipxx_yolov8.mp4"
out_video_path = DATA_OUTPUTS_DIR / f"segmented_clip{clip_id}_yolov8.mp4"

# Inizializza VideoWriter con risoluzione del primo frame
h, w = segmented_frames[0].shape[:2]
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(out_video_path), fourcc, fps, (w, h))

written_names = []  # per la lista dinamica dei frame inseriti
recon_handle = display(Markdown("_Inizio ricostruzione video…_"), display_id=True)

for i, frame in tqdm(list(enumerate(segmented_frames, start=1)), desc="Ricostruzione video YOLOv8", unit="frame"):
    # Scrive il frame nel video (convertendo da RGB a BGR per OpenCV)
    writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    # Aggiorna lista dinamica (ultimi 10)
    name = f"frame{i:02d}_clip{clip_id}_segmented_yolov8.png"
    written_names.append(name)
    last_w = written_names[-10:]
    recon_handle.update(Markdown(
        "**Frame inseriti nel video (ultimi 10):**  \n" + "<br>".join(last_w) + f"<br><br>Totale: **{len(written_names)}**"
    ))

writer.release()

# ---------- (7) Report finale ----------

fps_effettivo = len(segmented_frames) / elapsed_time

print(
    
    "✅ Segmentazione YOLOv8 avvenuta con successo.\n"
    "\n"
    "🔹 ---------------- Caratteristiche modello / architettura utilizzata ----------------------\n"
    f"- Cartella notebook eseguito: {BASE_DIR}\n"
    f"- Tipo architettura YOLO: {yolo_type}\n"
    f"- Modello YOLOv8 utilizzato: {model_path.name} (cartella: {MODELS_DIR})\n"
    
    "\n🔹 ---------------- Caratteristiche dell'input dell'inferenza---------------------------------\n"
    f"- Segmentazione relativa a: clip{clip_id}.mp4\n"
    f"- FPS della clip{clip_id}: {fps} fps\n"
    f"- Durata della clip {clip_id}: {len(segmented_frames)/25:.3f} secondi\n"
    f"- Cartella frames estratti da clip{clip_id}: {frames_dir}\n"
    f"- Risoluzione singoli frames: {w}x{h} pixel\n"
  
    "\n🔹 ---------------- Caratteristiche dell'output dell'inferenza---------------------------------\n"
    f"- Cartella frames segmentati (PNG): {seg_frames_dir}\n"
    f"- Cartella clip segmentata ricostruita (MP4): {out_video_path}\n"

    "\n🔹 ---------------- Analisi inferenza---------------------------------\n"
    f"- Numero totale di frame segmentati: {len(segmented_frames)} frames\n"
    f"- Tempo totale di inferenza: {elapsed_time:.3f} secondi\n"
    f"- Velocità media di inferenza: {fps_effettivo:.2f} frames/secondo\n"
    f"- Tempo di inferenza medio per ciascun frame: {1/fps_effettivo:.2f} secondi/frame\n"
    f"- Allocazione tempo unitario di inferenza (calcolato come tempo clip video/tempo di inferenza totale): {(len(segmented_frames)/25)/elapsed_time:.3f} secondi di clip{clip_id} elaborati ogni secondo di inferenza\n"
)

# ============================================
# Fine notebook YOLOv8
# ============================================

Torch: 2.7.1+cu118 | CUDA: True | Device: 0
GPU usata: NVIDIA GeForce GTX 1070


**Frame analizzati (ultimi 10):**  
frame136_clip08.png<br>frame137_clip08.png<br>frame138_clip08.png<br>frame139_clip08.png<br>frame140_clip08.png<br>frame141_clip08.png<br>frame142_clip08.png<br>frame143_clip08.png<br>frame144_clip08.png<br>frame145_clip08.png<br><br>Totale: **145**

Completamento inferenza YOLOv8 sui frames: 100%|██████████| 145/145 [01:35<00:00,  1.53frame/s]


**Frame inseriti nel video (ultimi 10):**  
frame136_clip08_segmented_yolov8.png<br>frame137_clip08_segmented_yolov8.png<br>frame138_clip08_segmented_yolov8.png<br>frame139_clip08_segmented_yolov8.png<br>frame140_clip08_segmented_yolov8.png<br>frame141_clip08_segmented_yolov8.png<br>frame142_clip08_segmented_yolov8.png<br>frame143_clip08_segmented_yolov8.png<br>frame144_clip08_segmented_yolov8.png<br>frame145_clip08_segmented_yolov8.png<br><br>Totale: **145**

Ricostruzione video YOLOv8: 100%|██████████| 145/145 [00:02<00:00, 56.71frame/s]

✅ Segmentazione YOLOv8 avvenuta con successo.

🔹 ---------------- Caratteristiche modello / architettura utilizzata ----------------------
- Cartella notebook eseguito: c:\Users\ricks\Desktop\ProgettoDL\Football_match_segmentation\1_yolov8
- Tipo architettura YOLO: yolov8x-seg
- Modello YOLOv8 utilizzato: yolov8x-seg.pt (cartella: c:\Users\ricks\Desktop\ProgettoDL\Football_match_segmentation\1_yolov8\models)

🔹 ---------------- Caratteristiche dell'input dell'inferenza---------------------------------
- Segmentazione relativa a: clip08.mp4
- FPS della clip08: 25.0 fps
- Durata della clip 08: 5.800 secondi
- Cartella frames estratti da clip08: C:\Users\ricks\Desktop\ProgettoDL\Football_match_segmentation\data_inputs\extracted_frames_clip08
- Risoluzione singoli frames: 1920x1080 pixel

🔹 ---------------- Caratteristiche dell'output dell'inferenza---------------------------------
- Cartella frames segmentati (PNG): c:\Users\ricks\Desktop\ProgettoDL\Football_match_segmentation\1_yolov8\da